In [1]:
import  torch
from torchvision import transforms
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import os
from PIL import Image

#### Drawing the confusion matrix

In [2]:
def draw_conf_matrix_clas_report(y_true, y_pred, name_labels='', name_model=''):

    c_m = confusion_matrix(y_true, y_pred)
    conf_matrix = pd.DataFrame(c_m, name_labels, name_labels)

    plt.figure(figsize = (9,9))

    group_counts = ['{0:0.0f}'.format(value) for value in
                  conf_matrix.values.flatten()]
    group_percentages = ['{0:.1%}'.format(value) for value in
                        conf_matrix.div(np.sum(conf_matrix, axis=1), axis=0).values.flatten()]

    labels = ['{}\n{}'.format(v1,v2) for v1,v2 in zip(group_counts, group_percentages)]

    labels = np.asarray(labels).reshape(c_m.shape)
    sns.set(font_scale=1.8)
    chart = sns.heatmap(conf_matrix,
              cbar=False ,
              annot=labels,
              square=True,
              fmt='',
              annot_kws={ 'size': 18},
              cmap="Blues",
              )
    chart.set_xticklabels(name_labels)
    chart.set_yticklabels(name_labels, rotation=360, verticalalignment='center')
    plt.savefig('confusion_matrix_{}.png'.format(name_model), bbox_inches='tight', pad_inches=0)

#### Pre-processing

In [3]:
def tf_processing(fp):
    def preprocess_input(x):
        x_temp = np.copy(x)
        x_temp = x_temp[..., ::-1]
        x_temp[..., 0] -= 91.4953
        x_temp[..., 1] -= 103.8827
        x_temp[..., 2] -= 131.0912
        return x_temp

    def get_img_tf(path):
        img = tf.keras.utils.load_img(
                    path,
                    target_size=(224,224),
                )
        
        img = tf.keras.utils.img_to_array(img)
        img = preprocess_input(img)
        img = np.array([img])
        return img

    return get_img_tf(fp)


def pth_processing(fp):
    class PreprocessInput(torch.nn.Module):
        def init(self):
            super(PreprocessInput, self).init()

        def forward(self, x):
            x = x.to(torch.float32)
            x = torch.flip(x, dims=(0,))
            x[0, :, :] -= 91.4953
            x[1, :, :] -= 103.8827
            x[2, :, :] -= 131.0912
            return x

    def get_img_torch(path):
        img = Image.open(path)
        img = img.resize((224, 224), Image.Resampling.NEAREST)
        
        ttransform = transforms.Compose([
            transforms.PILToTensor(),
            PreprocessInput()
        ])
        
        img = ttransform(img)
        img = torch.unsqueeze(img, 0)
        return img
    return get_img_torch(fp)

#### Loading a model that better predicts Neutral, Happiness, Surprise, and Anger classes 

In [4]:
name = '0_66_49_wo_gl'

# tf
tf_model = tf.keras.models.load_model('models_EmoAffectnet/weights_{0}.h5'.format(name))

# torch
pth_model = torch.jit.load('models_EmoAffectnet/torchscript_model_{0}.pth'.format(name))

#### Test tf_model 0_66_49_wo_gl¶

In [5]:
names_list = ['subDirectory_filePath','face_x','face_y','face_width','face_height','facial_landmarks','expression','valence','arousal']
dict_name_labels = {0: 'Neutral', 1: 'Happiness', 2: 'Sadness', 3: 'Surprise', 4: 'Fear', 5: 'Disgust', 6: 'Anger'}

path_df = 'C:/Work/AffectNet/Manually_Annotated_file_lists/validation.csv'
path_imgs = 'C:/Work/Databases/Manually_Annotated_Images_valid/'

df_valid = pd.read_csv(path_df, names = names_list)
df_valid = df_valid[~df_valid.expression.isin([7,8,9,10])] 
df_valid = df_valid.reset_index(drop=True)

labels = df_valid.expression.values.tolist()

prob_tf = []

for curr_path in tqdm(df_valid.subDirectory_filePath.values.tolist()):
    path_img = os.path.join(path_imgs,curr_path)
    img = tf_processing(path_img)
    prob = tf_model(img, training=False).numpy()
    cl = np.argmax(prob)
    prob_tf.append(cl)

name_labels = ['NE', 'HA', 'SA', 'SU', 'FE', 'DI', 'AN']
# draw_conf_matrix_clas_report(labels, prob_tf, name_labels, 'AffectNet_valid_tf_1')
print(classification_report(labels, prob_tf, target_names=name_labels, digits=3))

100%|██████████████████████████████████████████████████████████████████████████████| 3500/3500 [02:09<00:00, 27.06it/s]

              precision    recall  f1-score   support

          NE      0.532     0.694     0.602       500
          HA      0.780     0.886     0.830       500
          SA      0.680     0.634     0.656       500
          SU      0.614     0.632     0.623       500
          FE      0.715     0.596     0.650       500
          DI      0.749     0.598     0.665       500
          AN      0.636     0.614     0.625       500

    accuracy                          0.665      3500
   macro avg      0.672     0.665     0.664      3500
weighted avg      0.672     0.665     0.664      3500



#### Test torch_model 0_66_49_wo_gl

In [6]:
prob_torch = []

pth_model.eval()
with torch.no_grad():
    for curr_path in tqdm(df_valid.subDirectory_filePath.values.tolist()):
        path_img = os.path.join(path_imgs,curr_path)
        img = pth_processing(path_img)
        output = torch.nn.functional.softmax(pth_model(img), dim=1).detach().numpy()
        cl = np.argmax(output)
        prob_torch.append(cl)

name_labels = ['NE', 'HA', 'SA', 'SU', 'FE', 'DI', 'AN']
# draw_conf_matrix_clas_report(labels, prob_torch, name_labels, 'AffectNet_valid_torch_1')
print(classification_report(labels, prob_torch, target_names=name_labels, digits=3))

100%|██████████████████████████████████████████████████████████████████████████████| 3500/3500 [02:51<00:00, 20.44it/s]

              precision    recall  f1-score   support

          NE      0.532     0.694     0.602       500
          HA      0.780     0.886     0.830       500
          SA      0.683     0.634     0.658       500
          SU      0.614     0.632     0.623       500
          FE      0.715     0.596     0.650       500
          DI      0.749     0.598     0.665       500
          AN      0.635     0.616     0.625       500

    accuracy                          0.665      3500
   macro avg      0.673     0.665     0.665      3500
weighted avg      0.673     0.665     0.665      3500



#### Loading a model that better predicts Sadness, Surprise, Fear, and Disgust classes 

In [7]:
name = '0_66_37_wo_gl'

# tf
tf_model = tf.keras.models.load_model('models_EmoAffectnet/weights_{0}.h5'.format(name))

# torch
pth_model = torch.jit.load('models_EmoAffectnet/torchscript_model_{0}.pth'.format(name))

#### Test tf_model 0_66_37_wo_gl¶

In [8]:
prob_tf = []

for curr_path in tqdm(df_valid.subDirectory_filePath.values.tolist()):
    path_img = os.path.join(path_imgs,curr_path)
    img = tf_processing(path_img)
    prob = tf_model(img, training=False).numpy()
    cl = np.argmax(prob)
    prob_tf.append(cl)

name_labels = ['NE', 'HA', 'SA', 'SU', 'FE', 'DI', 'AN']
# draw_conf_matrix_clas_report(labels, prob_tf, name_labels, 'AffectNet_valid_tf_2')
print(classification_report(labels, prob_tf, target_names=name_labels, digits=3))

100%|██████████████████████████████████████████████████████████████████████████████| 3500/3500 [02:06<00:00, 27.59it/s]


              precision    recall  f1-score   support

          NE      0.533     0.680     0.598       500
          HA      0.779     0.880     0.826       500
          SA      0.670     0.642     0.656       500
          SU      0.616     0.632     0.624       500
          FE      0.714     0.600     0.652       500
          DI      0.724     0.620     0.668       500
          AN      0.648     0.592     0.619       500

    accuracy                          0.664      3500
   macro avg      0.669     0.664     0.663      3500
weighted avg      0.669     0.664     0.663      3500



#### Test torch_model 0_66_37_wo_gl¶

In [9]:
prob_torch = []

pth_model.eval()
with torch.no_grad():
    for curr_path in tqdm(df_valid.subDirectory_filePath.values.tolist()):
        path_img = os.path.join(path_imgs,curr_path)
        img = pth_processing(path_img)
        output = torch.nn.functional.softmax(pth_model(img), dim=1).detach().numpy()
        cl = np.argmax(output)
        prob_torch.append(cl)

name_labels = ['NE', 'HA', 'SA', 'SU', 'FE', 'DI', 'AN']
# draw_conf_matrix_clas_report(labels, prob_torch, name_labels, 'AffectNet_valid_torch_2')
print(classification_report(labels, prob_torch, target_names=name_labels, digits=3))

100%|██████████████████████████████████████████████████████████████████████████████| 3500/3500 [02:54<00:00, 20.02it/s]

              precision    recall  f1-score   support

          NE      0.533     0.680     0.598       500
          HA      0.779     0.880     0.826       500
          SA      0.670     0.642     0.656       500
          SU      0.616     0.632     0.624       500
          FE      0.714     0.600     0.652       500
          DI      0.726     0.620     0.669       500
          AN      0.648     0.594     0.620       500

    accuracy                          0.664      3500
   macro avg      0.670     0.664     0.664      3500
weighted avg      0.670     0.664     0.664      3500

